## XGBoost

In [2]:
%sh

pip install xgboost==0.90

Requirement already satisfied: xgboost==0.90 in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (0.90)
Requirement already satisfied: scipy in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from xgboost==0.90) (1.2.1)
Requirement already satisfied: numpy in /databricks/conda/envs/databricks-ml/lib/python3.7/site-packages (from xgboost==0.90) (1.16.2)

In [3]:
import pandas as pd
import xgboost as xgb


In [5]:
dbutils.library.restartPython()

In [6]:

# Import libraries

from sklearn import tree

from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.linear_model import LinearRegression,BayesianRidge,Ridge, Lasso, ElasticNet
from sklearn.metrics import mean_squared_error,r2_score
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeRegressor
from statsmodels.api import OLS
import matplotlib.pyplot as plt
import matplotlib.ticker as plticker
import numpy as np
import pandas as pd
#import pydotplus
from re import sub
import scipy.stats as sci
import seaborn as sns
 
# Set visualization prefrences 
sns.set(font_scale=1.5, style="darkgrid")
pd.set_option('display.max_columns', None)

/databricks/python/lib/python3.7/site-packages/patsy/constraint.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 from collections import Mapping
/databricks/python/lib/python3.7/site-packages/statsmodels/base/wrapper.py:100: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
 defaults=argspec[3])

In [7]:
df = spark.read.parquet('/databricks/driver/spark_merged_emp_modeling1.parquet')
#df.show(10)
#df= df.toPandas()

In [8]:
non_features = ['county','state','POP_ESTIMATE_2018','total_state_pop', 
                'pop_fraction']

to_be_normalzied_features = ['cases','deaths','All_Specialties_AAMC',
                             'Total_nurse_practitioners_2019',
                             'Total_physician_assistants_2019',
                             'Total_Hospitals_2019',
                             'Total_Primary_Care_Physicians_2019',
                             'Surgery_specialists_2019',
                             'Emergency_Medicine_specialists_2019',
                             'Total_Specialist_Physicians_2019',
                             'ICU_Beds',
                             ]



in_percentage = ['Percentage_of_Active_Physicians_Who_Are_Female_2018_AAMC',
                 'Percentage_of_Active_Physicians_Who_Are_Intertiol_Medical_Graduates_IMGs_2018_AAMC',
                 'Percentage_of_Active_Physicians_Who_Are_Age_60_or_Older_2018_AAMC',
                 'Percentage_Change_in_Student_Enrollment_at_MD_and_DO_Schools_2008_2018_AAMC',
                 'Percentage_of_MD_Students_Matriculating_In_State_AY_2018_2019_AAMC',
                 'Percentage_of_Residents_in_ACGME_Programs_Who_Are_IMGs_as_of_December_31_2018_AAMC',
                 'Percent_Change_in_Residents_and_Fellows_in_ACGME_Accredited_Programs_2008_2018_AAMC',
              'Percentage_of_Physicians_Retained_in_State_from_Undergraduate_Medical_Education_UME_2018_AAMC',
                 ]


patient = ['Adult_smoking_percentage',	'Adult_obesity_percentage',	'Excessive_drinking_percentage']



heathcare_county =  ['cases', 'deaths', 'All_Specialties_AAMC', 'Total_nurse_practitioners_2019',	'Total_physician_assistants_2019',	'Total_Hospitals_2019',	'Total_Primary_Care_Physicians_2019',	'Surgery_specialists_2019',	       'Emergency_Medicine_specialists_2019',	'Total_Specialist_Physicians_2019',	'ICU_Beds',	'Health_Behaviors_rank',	'Clinical_Care_rank']



socio_economic_county = [ 'POP_ESTIMATE_2018',	'pop_fraction',	'Length_of_Life_rank',	'Quality_of_Life_rank',		'Social_Economic_Factors_rank',	'Physical_Environment_rank',	'Population_per_sq_mile',	'House_per_sq_mile',  'Unemployed_2019', 'Unemployment_rate_2019',	'Median_Household_Income_2018',	'Med_HH_Income_Percent_of_State_Total_2018' ]

healthcare_state =[	'Active_Physicians_per_100000_Population_2018_AAMC',	'Total_Active_Patient_Care_Physicians_per_100000_Population_2018_AAMC',	'Active_Primary_Care_Physicians_per_100000_Population_2018_AAMC',	'Active_Patient_Care_Primary_Care_Physicians_per_100000_Population_2018_AAMC',	'Active_General_Surgeons_per_100000_Population_2018_AAMC',	'Active_Patient_Care_General_Surgeons_per_100000_Population_2018_AAMC',	'Percentage_of_Active_Physicians_Who_Are_Female_2018_AAMC',	'Percentage_of_Active_Physicians_Who_Are_Intertiol_Medical_Graduates_IMGs_2018_AAMC',	'Percentage_of_Active_Physicians_Who_Are_Age_60_or_Older_2018_AAMC',	'MD_and_DO_Student_Enrollment_per_100000_Population_AY_2018_2019_AAMC',	'Student_Enrollment_at_Public_MD_and_DO_Schools_per_100000_Population_AY_2018_2019_AAMC',	'Percentage_Change_in_Student_Enrollment_at_MD_and_DO_Schools_2008_2018_AAMC',	'Percentage_of_MD_Students_Matriculating_In_State_AY_2018_2019_AAMC',	'Total_Residents_Fellows_in_ACGME_Programs_per_100000_Population_as_of_December_31_2018_AAMC',	'Total_Residents_Fellows_in_Primary_Care_ACGME_Programs_per_100000_Population_as_of_Dec_31_2018_AAMC',	'Percentage_of_Residents_in_ACGME_Programs_Who_Are_IMGs_as_of_December_31_2018_AAMC',	'Ratio_of_Residents_and_Fellows_GME_to_Medical_Students_UME_AY_2017_2018_AAMC',	'Percent_Change_in_Residents_and_Fellows_in_ACGME_Accredited_Programs_2008_2018_AAMC',	'Percentage_of_Physicians_Retained_in_State_from_Undergraduate_Medical_Education_UME_2018_AAMC','State_Local_Government_hospital_beds_per_1000_people_2019',	'Non_profit_hospital_beds_per_1000_people_2019',	'For_profit_hospital_beds_per_1000_people_2019',	'Total_hospital_beds_per_1000_people_2019',	'Share_of_Tests_with_Positive_COVID_19_Results',	'Number_of_Tests_with_Results_per_1000_Population']


socio_economic_state = ['total_state_pop']



In [9]:
display(df)

county,cases,deaths,POP_ESTIMATE_2018,All_Specialties_AAMC,Total_nurse_practitioners_2019,Total_physician_assistants_2019,Total_Hospitals_2019,Total_Primary_Care_Physicians_2019,Surgery_specialists_2019,Emergency_Medicine_specialists_2019,Total_Specialist_Physicians_2019,ICU_Beds,pop_fraction,Length_of_Life_rank,Quality_of_Life_rank,Health_Behaviors_rank,Clinical_Care_rank,Social_Economic_Factors_rank,Physical_Environment_rank,Adult_smoking_percentage,Adult_obesity_percentage,Excessive_drinking_percentage,Population_per_sq_mile,House_per_sq_mile,Unemployed_2019,Unemployment_rate_2019,Median_Household_Income_2018,Med_HH_Income_Percent_of_State_Total_2018
Lee,7.117022517792556,0.5833625014584063,17142.0,2.2945925621281065,0.4171807667133357,0.20632844143040488,0.013571651557577879,1.257836399486641,0.1581392439621981,0.14319075819624313,1.2690477639715319,0.0,0.001685835,41.0,46.0,44.0,41.0,42.0,41.0,21.0,40.0,13.0,46.9,19.0,266.0,4.0,33199.0,63.29767965
Lexington,1.667615716261287,0.08473657094823613,295032.0,2.2945925621627485,0.4171807668320725,0.20632844145719786,0.013571651532715095,1.2578363994414166,0.158139243980314,0.14319075822283683,1.2690477636324196,0.18642045608611948,0.029015011,9.0,10.0,11.0,6.0,3.0,7.0,16.0,32.0,19.0,375.4,163.0,3510.0,2.3,60627.0,115.5922897
Marion,1.5464415735043011,0.12887013112535842,31039.0,2.294592562260382,0.41718076677728017,0.20632844144463416,0.013571651535165435,1.2578363993685362,0.15813924395115822,0.14319075820741647,1.2690477637810496,0.32217532781339603,0.00305254,40.0,45.0,41.0,44.0,43.0,19.0,20.0,44.0,14.0,67.6,30.6,564.0,4.3,34365.0,65.52079163
Marlboro,2.272899462080461,0.0,26398.0,2.294592562315327,0.41718076672475185,0.20632844143495718,0.013571651526630805,1.2578363993484354,0.15813924395787562,0.1431907582013789,1.2690477638457458,0.0,0.002596119,43.0,39.0,40.0,39.0,45.0,44.0,21.0,39.0,15.0,60.3,25.2,444.0,4.8,33534.0,63.93639536
McCormick,0.7438894792773645,0.10626992561105207,9410.0,2.2945925621679066,0.41718076673751325,0.206328441445271,0.013571651540913922,1.2578363995749204,0.15813924399574922,0.14319075823591926,1.2690477640807651,0.0,9.25429E-4,46.0,25.0,9.0,4.0,34.0,13.0,17.0,34.0,14.0,28.5,15.2,104.0,3.1,44683.0,85.19323533
Newberry,0.8047767393561787,0.02596053997923157,38520.0,2.294592562305296,0.41718076661474557,0.2063284414330218,0.013571651531671856,1.2578363995327102,0.1581392439771547,0.14319075822949118,1.269047763759086,0.181723779854621,0.003788261,10.0,21.0,23.0,21.0,15.0,16.0,18.0,36.0,16.0,59.5,28.4,514.0,2.7,48029.0,91.57276592
Oconee,0.49761400464439737,0.0,78374.0,2.2945925625845303,0.41718076670834714,0.2063284414474188,0.01357165152984408,1.257836399443693,0.15813924400949295,0.14319075815959373,1.2690477637992192,0.15311200142904535,0.007707715,16.0,4.0,14.0,12.0,12.0,15.0,17.0,32.0,16.0,118.6,61.9,971.0,2.8,50529.0,96.33930104
Orangeburg,1.2193158027929234,0.023005958543262706,86934.0,2.294592562173603,0.41718076667356846,0.20632844146133847,0.013571651528745943,1.2578363988773091,0.15813924390917247,0.14319075816136378,1.2690477638208295,0.23005958543262706,0.00854955,34.0,38.0,27.0,26.0,38.0,37.0,17.0,41.0,13.0,83.6,38.4,1655.0,4.8,37134.0,70.80020591
Pickens,0.6803428928179802,0.008004034033152708,124937.0,2.294592562651576,0.41718076670642007,0.20632844145449308,0.013571651536374333,1.257836399145169,0.1581392439389452,0.14319075822214394,1.2690477640730926,0.12006051049729063,0.012286967,12.0,8.0,6.0,7.0,9.0,33.0,17.0,32.0,18.0,240.2,103.2,1604.0,2.8,48794.0,93.03132567
Richland,2.814924163482691,0.142314075103238,414576.0,2.294592562280499,0.4171807666145652,0.20632844144378837,0.013571651533132646,1.2578363993574158,0.15813924397456677,0.1431907582204469,1.2690477637393385,0.5089537262166648,0.040771601,11.0,14.0,17.0,3.0,21.0,27.0,16.0,33.0,19.0,507.9,213.6,5579.0,2.8,52611.0,100.3088715


In [10]:
#df = df.drop("county")


In [11]:
df.columns

Out[6]: ['county',
 'cases',
 'deaths',
 'POP_ESTIMATE_2018',
 'All_Specialties_AAMC',
 'Total_nurse_practitioners_2019',
 'Total_physician_assistants_2019',
 'Total_Hospitals_2019',
 'Total_Primary_Care_Physicians_2019',
 'Surgery_specialists_2019',
 'Emergency_Medicine_specialists_2019',
 'Total_Specialist_Physicians_2019',
 'ICU_Beds',
 'pop_fraction',
 'Length_of_Life_rank',
 'Quality_of_Life_rank',
 'Health_Behaviors_rank',
 'Clinical_Care_rank',
 'Social_Economic_Factors_rank',
 'Physical_Environment_rank',
 'Adult_smoking_percentage',
 'Adult_obesity_percentage',
 'Excessive_drinking_percentage',
 'Population_per_sq_mile',
 'House_per_sq_mile',
 'Unemployed_2019',
 'Unemployment_rate_2019',
 'Median_Household_Income_2018',
 'Med_HH_Income_Percent_of_State_Total_2018']

In [12]:
df.where( df['pop_fraction'].isNull() ).count()

Out[7]: 30

In [13]:
df = df.drop("pop_fraction")

In [14]:
#cols = df.columns

In [15]:
###One-Hot Encoding
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler

categoricalColumns = ["county"]
stages = [] # stages in our Pipeline
for categoricalCol in categoricalColumns:
  # Category Indexing with StringIndexer
  stringIndexer = StringIndexer(inputCol=categoricalCol, outputCol=categoricalCol+"Index")
  # Use OneHotEncoder to convert categorical variables into binary SparseVectors
  encoder = OneHotEncoder(inputCol=categoricalCol+"Index", outputCol=categoricalCol+"classVec")
  # Add stages.  These are not run here, but will run all at once later 
  stages += [stringIndexer, encoder]

In [16]:
# Convert label into label indices using the StringIndexer
label_stringIdx = StringIndexer(inputCol = "deaths", outputCol = "label")
stages += [label_stringIdx]

In [17]:
# Transform all features into a vector using VectorAssembler
numericCols = ['cases',
 'POP_ESTIMATE_2018',
 'All_Specialties_AAMC',
 'Total_nurse_practitioners_2019',
 'Total_physician_assistants_2019',
 'Total_Hospitals_2019',
 'Total_Primary_Care_Physicians_2019',
 'Surgery_specialists_2019',
 'Emergency_Medicine_specialists_2019',
 'Total_Specialist_Physicians_2019',
 'ICU_Beds',
 'Length_of_Life_rank',
 'Quality_of_Life_rank',
 'Health_Behaviors_rank',
 'Clinical_Care_rank',
 'Social_Economic_Factors_rank',
 'Physical_Environment_rank',
 'Adult_smoking_percentage',
 'Adult_obesity_percentage',
 'Excessive_drinking_percentage',
 'Population_per_sq_mile',
 'House_per_sq_mile',
 'Unemployed_2019',
 'Unemployment_rate_2019',
 'Median_Household_Income_2018',
 'Med_HH_Income_Percent_of_State_Total_2018']
assemblerInputs = list(map(lambda c: c + "classVec", categoricalColumns)) + numericCols
#assemblerInputs = map(lambda c: c + "classVec", categoricalColumns) + numericCols
assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
stages += [assembler]

from pyspark.ml.feature import  VectorAssembler
from pyspark.sql.functions import array 

assembler =   VectorAssembler()\
                  .setInputCols(Array('Adult_smoking_percentage',	'Adult_obesity_percentage',	'Excessive_drinking_percentage', 'deaths', 'All_Specialties_AAMC',                          'Total_nurse_practitioners_2019',	'Total_physician_assistants_2019',	'Total_Hospitals_2019',	'Total_Primary_Care_Physicians_2019',	'Surgery_specialists_2019',	       'Emergency_Medicine_specialists_2019',	'Total_Specialist_Physicians_2019',	'ICU_Beds',	'Health_Behaviors_rank',	'Clinical_Care_rank', 'POP_ESTIMATE_2018',	'pop_fraction',	'Length_of_Life_rank',	'Quality_of_Life_rank',		'Social_Economic_Factors_rank',	'Physical_Environment_rank',	'Population_per_sq_mile',	'House_per_sq_mile',  'Unemployed_2019', 'Unemployment_rate_2019',	'Median_Household_Income_2018',	'Med_HH_Income_Percent_of_State_Total_2018', 'Active_Physicians_per_100000_Population_2018_AAMC',	'Total_Active_Patient_Care_Physicians_per_100000_Population_2018_AAMC',	'Active_Primary_Care_Physicians_per_100000_Population_2018_AAMC',	'Active_Patient_Care_Primary_Care_Physicians_per_100000_Population_2018_AAMC',	'Active_General_Surgeons_per_100000_Population_2018_AAMC',	'Active_Patient_Care_General_Surgeons_per_100000_Population_2018_AAMC',	'Percentage_of_Active_Physicians_Who_Are_Female_2018_AAMC',	'Percentage_of_Active_Physicians_Who_Are_Intertiol_Medical_Graduates_IMGs_2018_AAMC',	'Percentage_of_Active_Physicians_Who_Are_Age_60_or_Older_2018_AAMC',	'MD_and_DO_Student_Enrollment_per_100000_Population_AY_2018_2019_AAMC',	'Student_Enrollment_at_Public_MD_and_DO_Schools_per_100000_Population_AY_2018_2019_AAMC',	'Percentage_Change_in_Student_Enrollment_at_MD_and_DO_Schools_2008_2018_AAMC','Percentage_of_MD_Students_Matriculating_In_State_AY_2018_2019_AAMC','Total_Residents_Fellows_in_ACGME_Programs_per_100000_Population_as_of_December_31_2018_AAMC',	'Total_Residents_Fellows_in_Primary_Care_ACGME_Programs_per_100000_Population_as_of_Dec_31_2018_AAMC',	'Percentage_of_Residents_in_ACGME_Programs_Who_Are_IMGs_as_of_December_31_2018_AAMC',	'Ratio_of_Residents_and_Fellows_GME_to_Medical_Students_UME_AY_2017_2018_AAMC',	'Percent_Change_in_Residents_and_Fellows_in_ACGME_Accredited_Programs_2008_2018_AAMC',	'Percentage_of_Physicians_Retained_in_State_from_Undergraduate_Medical_Education_UME_2018_AAMC','State_Local_Government_hospital_beds_per_1000_people_2019',	'Non_profit_hospital_beds_per_1000_people_2019',	'For_profit_hospital_beds_per_1000_people_2019',	'Total_hospital_beds_per_1000_people_2019', 'Share_of_Tests_with_Positive_COVID_19_Results',	'Number_of_Tests_with_Results_per_1000_Population'))\
                  .setOutputCol("features")

In [19]:
# Create a Pipeline.
pipeline = Pipeline(stages=stages)
# Run the feature transformations.
#  - fit() computes feature statistics as needed.
#  - transform() actually transforms the features.
pipelineModel = pipeline.fit(df)
trainingData = pipelineModel.transform(df)

# Keep relevant columns
selectedcols = ["label", "features"] + cols
inputTrainingDF = trainingData.select(selectedcols)
#display(inputTrainingDF)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-2352528334825992> in <module> 
 ----> 1 model = pipeline . fit ( trainDF ) 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 --> 132 return self . _fit ( dataset ) 
 133 else : 
 134 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/pipeline.py in _fit (self, dataset) 
 107 dataset = stage . transform ( dataset ) 
 108 else : # must be an Estimator 
 --> 109 model = stage . fit ( dataset ) 
 110 transformers . append ( model ) 
 111 if i < indexOfLastEstimator : 

 /databricks/spark/python/pyspark/ml/base.py in fit (self, dataset, params) 
 130 return self . copy ( params ) . _fit ( dataset ) 
 131 else : 
 --> 132 return self . _fit ( dataset ) 
 133 else : 
 134 raise ValueError("Params must be either a param map or a list/tuple of param maps, "

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit (self, dataset) 
 293 
 294 def _fit ( self , dataset ) : 
 --> 295 java_model = self . _fit_java ( dataset ) 
 296 model = self . _create_model ( java_model ) 
 297 return self . _copyValues ( model ) 

 /databricks/spark/python/pyspark/ml/wrapper.py in _fit_java (self, dataset) 
 290 """
 291 self . _transfer_params_to_java ( ) 
 --> 292 return self . _java_obj . fit ( dataset . _jdf ) 
 293 
 294 def _fit ( self , dataset ) : 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o302.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 5 in stage 1795.0 failed 1 times, most recent failure: Lost task 5.0 in stage 1795.0 (TID 10890, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$4: (struct<countyclassVec:struct<type:tinyint,size:int,indices:array<int>,values:array<double>>,cases:double,POP_ESTIMATE_2018:double,All_Specialties_AAMC:double,Total_nurse_practitioners_2019:double,Total_physician_assistants_2019:double,Total_Hospitals_2019:double,Total_Primary_Care_Physicians_2019:double,Surgery_specialists_2019:double,Emergency_Medicine_specialists_2019:double,Total_Specialist_Physicians_2019:double,ICU_Beds:double,Length_of_Life_rank:double,Quality_of_Life_rank:double,Health_Behaviors_rank:double,Clinical_Care_rank:double,Social_Economic_Factors_rank:double,Physical_Environment_rank:double,Adult_smoking_percentage:double,Adult_obesity_percentage:double,Excessive_drinking_percentage:double,Population_per_sq_mile:double,House_per_sq_mile:double,Unemployed_2019:double,Unemployment_rate_2019:double,Median_Household_Income_2018:double,Med_HH_Income_Percent_of_State_Total_2018:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.sort_addToSorter_0$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.s

In [23]:
features = inputTrainingDF.select("features")
lables = inputTrainingDF.select("label")


In [24]:
X = np.asarray(map(lambda v: v[0].toArray(), features))
Y = np.asarray(map(lambda v: v[0], lables))

In [25]:
labelAndFeatures = inputTrainingDF.select("deaths", "label", "features").rdd.map(lambda row: [row[0], [row[1], row[2]]]).groupByKey().map(lambda v: trainXGbModel(v[0], list(v[1])))

In [26]:
X = np.asarray(map(lambda v: v[1].toArray(), labelAndFeatures))
Y = np.asarray(map(lambda v: v[0], labelAndFeatures))

--------------------------------------------------------------------------- 
 TypeError Traceback (most recent call last)
 <command-2352528334825998> in <module> 
 ----> 1 X = np . asarray ( map ( lambda v : v [ 1 ] . toArray ( ) , labelAndFeatures ) ) 
 2 Y = np . asarray ( map ( lambda v : v [ 0 ] , labelAndFeatures ) ) 

 TypeError : 'PipelinedRDD' object is not iterable

In [27]:
import xgboost as xgb
xgbXGBRegressor = xgb.XGBRegressor()


In [28]:
model =  xgbXGBRegressor.fit(X, Y)

--------------------------------------------------------------------------- 
 ValueError Traceback (most recent call last)
 <command-2352528334825997> in <module> 
 ----> 1 model = xgbXGBRegressor . fit ( X , Y ) 

 /databricks/python/lib/python3.7/site-packages/xgboost/sklearn.py in fit (self, X, y, sample_weight, eval_set, eval_metric, early_stopping_rounds, verbose, xgb_model, sample_weight_eval_set, callbacks) 
 358 missing=self.missing, nthread=self.n_jobs)
 359 else : 
 --> 360 trainDmatrix = DMatrix ( X , label = y , missing = self . missing , nthread = self . n_jobs ) 
 361 
 362 evals_result = { } 

 /databricks/python/lib/python3.7/site-packages/xgboost/core.py in __init__ (self, data, label, missing, weight, silent, feature_names, feature_types, nthread) 
 402 self . _init_from_csc ( data ) 
 403 elif isinstance ( data , np . ndarray ) : 
 --> 404 self . _init_from_npy2d ( data , missing , nthread ) 
 405 elif isinstance ( data , DataTable ) : 
 406 self . _init_from_dt ( data , nthread ) 

 /databricks/python/lib/python3.7/site-packages/xgboost/core.py in _init_from_npy2d (self, mat, missing, nthread) 
 472 """
 473 if len ( mat . shape ) != 2 : 
 --> 474 raise ValueError ( 'Input numpy.ndarray must be 2 dimensional' ) 
 475 # flatten the array by rows and ensure it is float32. 
 476 # we try to avoid data copies if possible (reshape returns a view when possible 

 ValueError : Input numpy.ndarray must be 2 dimensional

In [29]:
def trainXGbModel(partitionKey, labelAndFeatures):
  X = np.asarray(map(lambda v: v[1].toArray(), labelAndFeatures))
  Y = np.asarray(map(lambda v: v[0], labelAndFeatures))
  xgbXGBRegressor = xgb.XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)
  model =  xgbXGBRegressor.fit(X, Y)
  return [partitionKey, model]

In [30]:
xgbModels = inputTrainingDF.select("deaths", "label", "features").rdd.map(lambda row: [row[0], [row[1], row[2]]]).groupByKey().map(lambda v: trainXGbModel(v[0], list(v[1])))

In [31]:
xgbModels.take(1)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-2352528334825988> in <module> 
 ----> 1 xgbModels . take ( 1 ) 

 /databricks/spark/python/pyspark/rdd.py in take (self, num) 
 1399 
 1400 p = range ( partsScanned , min ( partsScanned + numPartsToTry , totalParts ) ) 
 -> 1401 res = self . context . runJob ( self , takeUpToNumLeft , p ) 
 1402 
 1403 items += res

 /databricks/spark/python/pyspark/context.py in runJob (self, rdd, partitionFunc, partitions, allowLocal) 
 1176 finally : 
 1177 os . remove ( filename ) 
 -> 1178 sock_info = self . _jvm . PythonRDD . runJob ( self . _jsc . sc ( ) , mappedRDD . _jrdd , partitions ) 
 1179 return list ( _load_from_socket ( sock_info , mappedRDD . _jrdd_deserializer ) ) 
 1180 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1255 answer = self . gateway_client . send_command ( command ) 
 1256 return_value = get_return_value(
 -> 1257 answer, self.gateway_client, self.target_id, self.name)
 1258 
 1259 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 61 def deco ( * a , ** kw ) : 
 62 try : 
 ---> 63 return f ( * a , ** kw ) 
 64 except py4j . protocol . Py4JJavaError as e : 
 65 s = e . java_exception . toString ( ) 

 /databricks/spark/python/lib/py4j-0.10.7-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 1780.0 failed 1 times, most recent failure: Lost task 2.0 in stage 1780.0 (TID 10825, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$4: (struct<countyclassVec:struct<type:tinyint,size:int,indices:array<int>,values:array<double>>,cases:double,POP_ESTIMATE_2018:double,All_Specialties_AAMC:double,Total_nurse_practitioners_2019:double,Total_physician_assistants_2019:double,Total_Hospitals_2019:double,Total_Primary_Care_Physicians_2019:double,Surgery_specialists_2019:double,Emergency_Medicine_specialists_2019:double,Total_Specialist_Physicians_2019:double,ICU_Beds:double,Length_of_Life_rank:double,Quality_of_Life_rank:double,Health_Behaviors_rank:double,Clinical_Care_rank:double,Social_Economic_Factors_rank:double,Physical_Environment_rank:double,Adult_smoking_percentage:double,Adult_obesity_percentage:double,Excessive_drinking_percentage:double,Population_per_sq_mile:double,House_per_sq_mile:double,Unemployed_2019:double,Unemployment_rate_2019:double,Median_Household_Income_2018:double,Med_HH_Income_Percent_of_State_Total_2018:double>) => struct<type:tinyint,size:int,indices:array<int>,values:array<double>>)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$13$$anon$1.hasNext(WholeStageCodegenExec.scala:640)
	at scala.collection.Iterator$$anon$11.hasNext(Iterator.scala:409)
	at org.apache.spark.api.python.SerDeUtil$AutoBatchedPickler.next(SerDeUtil.scala:158)
	at org.apache.spark.api.python.SerDeUtil$AutoBatchedPickler.next(SerDeUtil.scala:150)
	at scala.collection.Iterator$class.foreach(Iterator.scala:891)
	at org.apache.spark.api.python.SerDeUtil$AutoBatchedPickler.foreach(SerDeUtil.scala:150)
	at org.apache.spark.api.python.PythonRDD$.writeIteratorToStream(PythonRDD.scala:368)
	at org.apache.spark.api.python.PythonRunner$$anon$2.writeIteratorToStream(PythonRunner.scala:645)
	at org.apache.spark.api.python.BasePythonRunner$WriterTh

In [32]:
# Set regresssion models for testing
reg_models = list()

reg_models = [
    ('Linear regression',LinearRegression()),
    ('Bayesian Ridge regression', BayesianRidge()),
    ('Decision tree',DecisionTreeRegressor(max_depth=1500,random_state=0)),
    ('Random forest',RandomForestRegressor(max_depth=1500,random_state=0)),
    ('Gradient boosting',GradientBoostingRegressor(random_state=0)),
    ('Linear regression with Ridge regularization', Ridge()),
    ('Linear regression with Lasso regularization (L1 penalty)',Lasso(random_state=0)),
    ('Linear regression with ElasticNet regularization',ElasticNet(random_state=0))
     ]

In [33]:
# Train models and get test results.
results = pd.DataFrame(columns = ['Model','RMSE','R-squared'])
models = []

for i,model in enumerate(reg_models):
    trained_model, RMSE, R2 = regression_fit(model[1], X_train[train_vars], y_train, X_test[train_vars], y_test)
    results.loc[i] = [model[0],RMSE,R2]
    models.append(trained_model)

In [34]:
results.sort_values(by='R-squared', ascending=False)

In [35]:
mlp = MLPRegressor(hidden_layer_sizes=(5,5,5), activation='logistic', 
                   learning_rate='adaptive', solver='lbfgs', max_iter=2000, 
                   learning_rate_init=.001, random_state=10)

mlp.fit(X_train[train_vars],y_train)

predict_train = mlp.predict(X_train[train_vars])
predict_test = mlp.predict(X_test[train_vars])

In [36]:
# Calculate root-mean-square error (RMSE), indicative of diffrences between predicted and obseverd values
RMSE = np.sqrt(mean_squared_error(y_true = y_test, y_pred = predict_test))

# Calcualte R square, indicative of the percentage of variance of the predicted varialbe explained by the model
R2 = r2_score(y_test,predict_test)

print(RMSE,R2)
results.loc[i+1] = ['Multilayer Perceptron', RMSE, R2]


In [37]:
# Train a neural netowrk (MLP)

mlp = MLPRegressor(hidden_layer_sizes=(5,5,5), activation='logistic', 
                   learning_rate='adaptive', solver='lbfgs', max_iter=2000, 
                   learning_rate_init=.001, random_state=10)

mlp.fit(X_train[train_vars],y_train)

predict_train = mlp.predict(X_train[train_vars])
predict_test = mlp.predict(X_test[train_vars])

In [38]:
# Calculate root-mean-square error (RMSE), indicative of diffrences between predicted and obseverd values
RMSE = np.sqrt(mean_squared_error(y_true = y_test, y_pred = predict_test))

# Calcualte R square, indicative of the percentage of variance of the predicted varialbe explained by the model
R2 = r2_score(y_test,predict_test)

print(RMSE,R2)
results.loc[i+1] = ['Multilayer Perceptron', RMSE, R2]

In [39]:
# Plot an example tree from the Gradient boosting model
from IPython.display import Image
%matplotlib inline
from IPython.core.display import HTML 
from IPython.display import display
#from pyspark.ml.image import ImageSchema
sub_tree =  models[4].estimators_[10, 0]
dot_data = tree.export_graphviz(
    sub_tree,
    out_file=None, filled=True,
    rounded=True,  
    special_characters=True,
    proportion=True,
    feature_names=X_train[train_vars].columns
)

graph = pydotplus.graph_from_dot_data(dot_data)  
#Image(graph.create_png())
Image(graph.create_png()).show()

In [40]:
im = Image(graph.create_png()).visualize()
open('output.png', 'wb').write(im.data)